In [1]:
# Library Import
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import pandas as pd
import os

In [2]:
# Get current working directory
cwd = os.getcwd()
print(cwd)

/Users/uditanand/Desktop/Github/GooglePracticum/Data/Maize


In [3]:
# Import Names of counties from file
#maize_data_dir = cwd + '/Data/Maize/'
#IL_county_data = maize_data_dir + 'County_Names_IL.csv'

IL_county_data = 'County_Name_IL_updated.csv'
county_data = pd.read_csv(IL_county_data)

# rename Ag district to Ag_District
county_data.rename(columns={'Ag District': 'Ag_District'}, inplace=True)

# making all column names UPPER CASE --> inline with the weather data file
county_data.columns = map(str.upper, county_data.columns)

# making COUNTY, STATE, AG DISTRICT, COMMODITY data lower case
# CAN BE MADE FASTER WITH A LAMBDA FUNCTION
county_data.STATE = county_data.STATE.str.lower()
county_data.AG_DISTRICT = county_data.AG_DISTRICT.str.lower()
county_data.COUNTY = county_data.COUNTY.str.lower() 
county_data.COMMODITY = county_data.COMMODITY.str.lower()

print(county_data.head())

# Print name of unique districts
ag_district_list = list(county_data['AG_DISTRICT'].unique())
print(" \n Agricultural Districts: \n")
print(ag_district_list)

      STATE AG_DISTRICT      COUNTY COMMODITY     VALUE
0  illinois     central      mclean      corn  63008000
1  illinois     central       logan      corn  41005000
2  illinois     central       macon      corn  30430000
3  illinois        east    iroquois      corn  62773000
4  illinois        east  livingston      corn  54632000
 
 Agricultural Districts: 

['central', 'east', 'east southeast', 'northeast', 'northwest', 'southeast', 'southwest', 'west', 'west southwest']


In [4]:
# Group counties belonging to a particular agricultural district and store them in a dictionary

district_county_dict = {}

for district in ag_district_list:
    county_in_ag_district = county_data[(county_data['AG_DISTRICT'] == district)] 
    county_in_ag_district = county_in_ag_district['COUNTY']
    district_county_dict[district] = county_in_ag_district

# Print the dictionary, Structure {Region_1: County_1, County_2 ,... ; Region_2: County_1, County_2.. }
for key,values in district_county_dict.items():
    print(key)
    print(values)
    print("\n")

central
0    mclean
1     logan
2     macon
Name: COUNTY, dtype: object


east
3      iroquois
4    livingston
5     champaign
Name: COUNTY, dtype: object


east southeast
6     shelby
7      edgar
8    douglas
Name: COUNTY, dtype: object


northeast
9     la salle
10     de kalb
11      grundy
Name: COUNTY, dtype: object


northwest
12    bureau
13      ogle
14       lee
Name: COUNTY, dtype: object


southeast
15       white
16       wayne
17    gallatin
Name: COUNTY, dtype: object


southwest
18    washington
19      st clair
20       clinton
Name: COUNTY, dtype: object


west
21       knox
22    hancock
23     warren
Name: COUNTY, dtype: object


west southwest
24     sangamon
25    christian
26     macoupin
Name: COUNTY, dtype: object




In [5]:
# Import weather data file
#weather_data_dir = cwd + '/Data/Weather/'
#sample_weather_file = weather_data_dir + 'weather_data.csv'
sample_weather_file = 'weather_data.csv'
sample_weather_data = pd.read_csv(sample_weather_file)

print("\n Column Information \n")
print(sample_weather_data.columns.all)

FileNotFoundError: File b'weather_data.csv' does not exist

In [ ]:
# Append corresponding AGRICULTURAL DISTRICT to COUNTY
sample_weather_data['AG_DISTRICT'] = "Dummy"
sample_weather_data.drop('Unnamed: 0', axis=1, inplace=True)

# reorder the AG_DISTRICT to become the first column
colnames = sample_weather_data.columns.tolist()
colnames = colnames[-1:] + colnames[:-1]
sample_weather_data = sample_weather_data[colnames]

county_names_list = list(county_data['COUNTY'].unique())

for county in county_names_list:
    county_district = county_data[(county_data['COUNTY'] == county)] 
    county_district = county_district['AG_DISTRICT']
    
    selected_county = sample_weather_data[(sample_weather_data['COUNTY'] == county)]
    
    sample_weather_data.loc[sample_weather_data['COUNTY'] == county, 'AG_DISTRICT'] = county_district.values

sample_weather_data = sample_weather_data[sample_weather_data.AG_DISTRICT != 'Dummy']
sample_weather_data.loc[:,['AG_DISTRICT','COUNTY']].head()
sample_weather_data.head()

#print(sample_weather_data.columns.all)

#sample_weather_data.iloc[:,5:26]


In [ ]:
months = list(sample_weather_data['MONTH'].unique())
months

In [ ]:
groupby_ag_district = sample_weather_data.groupby(['YEAR', 'MONTH','AG_DISTRICT'])
sample_data = groupby_ag_district['NUMDAY_RAIN', 'NUM_DAYS_SNOW', 'NUM_DAYS_THNDRSTRM','NUM_DAYS_SNOW','NUM_DAYS_FOG','PRECIP'].sum()

# Writes to Excel
# writer = pd.ExcelWriter('pandas_simple.xlsx', engine='xlsxwriter')
# Convert the dataframe to an XlsxWriter Excel object.
# sample_data.to_excel(writer, sheet_name='Sheet1')
# Close the Pandas Excel writer and output the Excel file.
# writer.save()

#sample_data.iloc[:9]
#sample_data['AG_DISTRICT' == 'Dummy']
sample_data.iloc[:,:]

In [ ]:
# Precipitation is measured in inches.

start_year = min(sample_weather_data['YEAR'])
end_year = max(sample_weather_data['YEAR'])

# end year + 1 to get it to 2016
for year in range(start_year, end_year+1):
    groupby_yearly = sample_weather_data[sample_weather_data['YEAR'] == year].groupby(['AG_DISTRICT', 'YEAR'])
    groupby_yearly_ppt_data = groupby_yearly['PRECIP'].sum()
    
    print(groupby_yearly_ppt_data.head())
    print("\n")
    #groupby_yearly_ppt_data = groupby_yearly_ppt_data/3
    #print(groupby_yearly_ppt_data.head())
    #print("\n")
    
    # Calculate average precipitation for the given year
    size = groupby_yearly_ppt_data.count()
    average_precipitation = sum(groupby_yearly_ppt_data.iloc[:size].values)/(size)
    
    # Plotting Code
    precp_yearly_title = str(year) + ": Total Precipitation received across various districts (in inches)."
    line_label = "Average Precipitation for the year " + str(year) 
    
    plt.figure(figsize=(15,7))
    plt.suptitle(precp_yearly_title, fontsize=16, fontweight='bold')
    ax1 = groupby_yearly_ppt_data.plot(kind='bar', label ='Precipitation')
    ax2 = plt.hlines(average_precipitation,-2,10, colors='r', linestyles='solid', label=line_label)
    plt.xticks(rotation=45)
    plt.ylabel('Inches', fontsize=16)
    plt.xlabel('Agricultural Districts', fontsize=16)
    plt.legend()
    #plt.savefig("results.jpg")
    plt.show()
    
    # 2008 validation
    # http://www.isws.illinois.edu/atmos/statecli/climate-change/NE-IL-trends/rainfall.htm

In [ ]:
start_year = min(sample_weather_data['YEAR'])
end_year = max(sample_weather_data['YEAR'])

# end year + 1 to get it to 2016
for year in range(start_year, end_year+1):
    groupby_yearly = sample_weather_data[sample_weather_data['YEAR'] == year].groupby(['AG_DISTRICT', 'YEAR'])
    groupby_yearly_ppt_data = groupby_yearly['NUMDAY_RAIN', 'NUM_DAYS_SNOW', 'NUM_DAYS_THNDRSTRM'].sum()
    
    print(groupby_yearly_ppt_data.head())
    print("\n")

    # Plotting Code
    rainfall_yearly_title = str(year) + ": Total Rainfall, Snowfall, Thunderstorm received across various districts"

    ax = groupby_yearly_ppt_data.plot(kind='bar', figsize=(15, 7))
    ax.set_title(rainfall_yearly_title, fontsize=16, fontweight='bold')
    plt.xticks(rotation=45)
    plt.ylabel('Number of Days', fontsize=14)
    plt.xlabel('Agricultural Districts', fontsize=14)
    plt.legend()
    plt.show()
    #plt.savefig("results.jpg")

In [ ]:
start_year = min(sample_weather_data['YEAR'])
end_year = max(sample_weather_data['YEAR'])
end_year = end_year +1

#cols = ['YEAR','TMP_HIGH', 'TMP_AVG',
#       'TMP_LOW', 'DP_HIGH', 'DP_AVG', 'DP_LOW', 'HUM_HIGH', 'HUM_AVG',
#       'HUM_LOW', 'SEALVL_HIGH', 'SEALVL_AVG', 'SEALVL_LOW', 'VIS_HIGH',
#       'VIS_AVG', 'VIS_LOW', 'WIND_HIGH', 'WIND_AVG', 'PRECIP', 'NUMDAY_RAIN',
#       'NUM_DAYS_SNOW', 'NUM_DAYS_FOG', 'NUM_DAYS_THNDRSTRM']

#model = pd.DataFrame(columns = cols)
model = pd.DataFrame()

# end year + 1 to get it to 2016
for year in range(start_year, end_year+1):
    groupby_yearly = sample_weather_data[sample_weather_data['YEAR'] == year].groupby(['AG_DISTRICT', 'YEAR', 'MONTH'])
    groupby_yearly_ppt_data = groupby_yearly['TMP_HIGH', 'TMP_AVG',
       'TMP_LOW', 'DP_HIGH', 'DP_AVG', 'DP_LOW', 'HUM_HIGH', 'HUM_AVG',
       'HUM_LOW', 'SEALVL_HIGH', 'SEALVL_AVG', 'SEALVL_LOW', 'VIS_HIGH',
       'VIS_AVG', 'VIS_LOW', 'WIND_HIGH', 'WIND_AVG', 'PRECIP'].mean()
    groupby_yearly_ppt_data['NUMDAY_RAIN'] = groupby_yearly['NUMDAY_RAIN'].sum()
    groupby_yearly_ppt_data['NUM_DAYS_SNOW'] = groupby_yearly['NUM_DAYS_SNOW'].sum()
    groupby_yearly_ppt_data['NUM_DAYS_FOG'] = groupby_yearly['NUM_DAYS_FOG'].sum()
    groupby_yearly_ppt_data['NUM_DAYS_THNDRSTRM'] = groupby_yearly['NUM_DAYS_THNDRSTRM'].sum()
    model = model.append(groupby_yearly_ppt_data)
    
    #print(groupby_yearly_ppt_data)
    #print("\n")

print(model)
model.to_csv('model.csv')

In [ ]:
df = pd.read_csv('model.csv')
#print(df)

cols = [ 'YEAR', 'MONTH' , 'TMP_HIGH', 'TMP_AVG', 'TMP_LOW', 'DP_HIGH',
       'DP_AVG', 'DP_LOW', 'HUM_HIGH', 'HUM_AVG', 'HUM_LOW', 'SEALVL_HIGH',
       'SEALVL_AVG', 'SEALVL_LOW', 'VIS_HIGH', 'VIS_AVG', 'VIS_LOW',
       'WIND_HIGH', 'WIND_AVG', 'PRECIP', 'NUMDAY_RAIN', 'NUM_DAYS_SNOW',
       'NUM_DAYS_FOG', 'NUM_DAYS_THNDRSTRM']

x = []

for year in range(start_year, end_year):
    df_temp = df[df['YEAR'] == year]
    for m in months:
        df_new = df_temp[df_temp['MONTH'] == m]
        temp = list(df_new[['YEAR','TMP_HIGH', 'TMP_AVG', 'TMP_LOW', 'DP_HIGH',
       'DP_AVG', 'DP_LOW', 'HUM_HIGH', 'HUM_AVG', 'HUM_LOW', 'SEALVL_HIGH',
       'SEALVL_AVG', 'SEALVL_LOW', 'VIS_HIGH', 'VIS_AVG', 'VIS_LOW',
       'WIND_HIGH', 'WIND_AVG', 'PRECIP', 'NUMDAY_RAIN', 'NUM_DAYS_SNOW',
       'NUM_DAYS_FOG', 'NUM_DAYS_THNDRSTRM']].mean())
        temp.insert(1, m)
        x.append(temp)
        
final = pd.DataFrame(x, columns = cols)
final.insert(0, 'STATE', 'IL')
final.YEAR = final.YEAR.astype(int)
final.NUMDAY_RAIN = final.NUMDAY_RAIN.astype(int)
final.NUM_DAYS_SNOW = final.NUM_DAYS_SNOW.astype(int)
final.NUM_DAYS_FOG = final.NUM_DAYS_FOG.astype(int)
final.NUM_DAYS_THNDRSTRM = final.NUM_DAYS_THNDRSTRM.astype(int)
print (final)

final.to_csv('final_yearly.csv')

In [ ]:
final_yearly = pd.read_csv('final_yearly.csv')
final_yearly

price_yearly = pd.read_csv('IL_Corn_Yearly_Month.csv')
price_yearly['Period'] = pd.Categorical(price_yearly['Period'], ['JAN', 'FEB', 'MAR', 'ARP', 'MAY', 'JUN', 'JUL', 'AUG', 'SEP', 'OCT', 'NOV', 'DEC'])
ps = price_yearly[['Year', 'Period','Value']]

ps = ps.sort_values(['Year', 'Period'])
ps = ps.reset_index(drop=True)

#print (ps)
#print (ps['Value'])

final_yearly['Price'] = ps['Value']
print (final_yearly.head())
final_yearly.to_csv('input_model.csv')


In [ ]:
input_model = pd.read_csv('input_model.csv')
input_model.drop('Unnamed: 0', axis=1, inplace=True)
input_model.drop('Unnamed: 0.1', axis=1, inplace=True)
input_model